# Routing Topology: OSPF using FRRouting

This notebook is an example of how to create a FABRIC routing experiment topology comprising nodes at three different sites. Each site has a local layer 2 (Ethernet) network connecting a set of local nodes and one gateway router. The three gateway routers connect to each other and use the [FRRouting](https://frrouting.org/) protocol suite to deploy [OSPF](https://en.wikipedia.org/wiki/Open_Shortest_Path_First) dameons to propagate route updates across the topology.

You might be familiar with the [Quagga](https://www.quagga.net/) router suite.  FRRouting is based on Quagga but has a more active upstream community including many large companies working on cloud networking.


<img src="./figs/frr.png" width="90%"><br>


## Import the FABlib Library


In [ ]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();


## Create the Experiment Slice

The following creates private layer 2 networks on three sites including OSPF gateway routers that propagate routes across the topology. 


In [ ]:
slice_name = 'OSPF_Routing_Topology'

sites= fablib.get_random_sites(count=3, avoid=[])

print(f"Sites: {sites}")

router_base_name='router'
router_link_base_name='link'

node_base_name='node'
local_network_base_name='net_local'

site_node_count=2

In [ ]:

slice = fablib.new_slice(name=slice_name)

# Create Routers
routers = []
frr_user_data = {}
for i, site in enumerate(sites):
    router_name = f"{router_base_name}{site}"
    frr_user_data[router_name] = {}
    local_subnet = IPv4Network(f"192.168.{i+1}.0/24")
    local_gateway = local_subnet[1]
    
    router = slice.add_node(name=router_name, site=site, cores=8, ram=16, disk=100, image='docker_rocky_8')
    router.add_fabnet()
    
    # Add experiment nets
    iface_local = router.add_component(model='NIC_Basic', name='nic_local').get_interfaces()[0]
    iface_local.set_mode('config')
    
    local_net = slice.add_l2network(name=f'{local_network_base_name}{site}', subnet=local_subnet, gateway=local_gateway)
    local_net.add_interface(iface_local)
    iface_local.set_ip_addr(local_gateway)
    
    frr_user_data[router_name]['local_iface'] = iface_local.get_name()
    frr_user_data[router_name]['link_ifaces'] = []
    
    router.add_post_boot_upload_directory('node_tools','.')    
    router.add_post_boot_execute('node_tools/enable_docker.sh {{ _self_.image }}')
    router.add_post_boot_upload_directory('docker_containers','.')
    
    iface_template = f"{{{{ interfaces['{iface_local.get_name()}'].dev }}}}"
    
    router.add_post_boot_execute(f"./docker_containers/fabric_frrouting/node_tools/frr_config_docker.sh {iface_template} {local_gateway} '192.168.0.0'")    
    routers.append(router)
    


# Create Links between routers (ring)
links = []
for i, site in enumerate(sites):
    
    link_info = {}
    link_subnet = IPv4Network(f"192.168.10{i+1}.0/24")
    
    router1 = routers[i]
    router2 = routers[(i+1)%len(sites)]
    
    print(f"link: {router1.get_name()} -> {router2.get_name()}")
    
    #link_info['name'] = f'{router_link_base_name}{i+1}'
    link_info['name'] = f'{router1.get_site()}-{router2.get_site()}-{router_link_base_name}{i+1}'
    
    link = slice.add_l2network(name=link_info['name'], subnet=link_subnet)
    links.append(link)
    
    iface1 = router1.add_component(model='NIC_Basic', name=f'{router1.get_site()}-{router2.get_site()}').get_interfaces()[0]
    iface2 = router2.add_component(model='NIC_Basic', name=f'{router2.get_site()}-{router1.get_site()}').get_interfaces()[0]
    
    iface1.set_mode('config')
    iface2.set_mode('config')

    print(f"iface1: {iface1.get_name()}")
    link.add_interface(iface1)
    print(f"iface2: {iface2.get_name()}")
    link.add_interface(iface2)
    
    iface1.set_ip_addr(link_subnet[1])
    iface2.set_ip_addr(link_subnet[2])
        
    frr_user_data[router1.get_name()]['link_ifaces'].append(iface1.get_name()) 
    frr_user_data[router2.get_name()]['link_ifaces'].append(iface2.get_name()) 
    
    iface1_template = f"{{{{ interfaces['{iface1.get_name()}'].dev }}}}"
    iface2_template = f"{{{{ interfaces['{iface2.get_name()}'].dev }}}}"

    router1.add_post_boot_execute(f'./docker_containers/fabric_frrouting/node_tools/add_ospf_neighbor.sh {iface1_template} {link_subnet[1]} ')
    router2.add_post_boot_execute(f'./docker_containers/fabric_frrouting/node_tools/add_ospf_neighbor.sh {iface2_template} {link_subnet[2]} ')

# Set frr user_data
for router in slice.get_nodes():
    user_data = router.get_user_data()
    user_data['frr'] = frr_user_data[router.get_name()]
    router.set_user_data(user_data)
    
    all_devs_template = ''
    for iface in router.get_interfaces():
        all_devs_template +=  f" {{{{ interfaces['{iface.get_name()}'].dev }}}} "

    router.add_post_boot_execute(f'./docker_containers/fabric_frrouting/start.sh {all_devs_template} ')
    
slice_id = slice.submit()


In [ ]:
for i, site in enumerate(sites):
    print(f"Adding nodes to {site}")
    print(f"update_count: {slice.update_count}, update_topology_count: {slice.update_topology_count}, update_slivers_count: {slice.update_slivers_count},  update_slice_count: {slice.update_slice_count}")
    for node_num in range(site_node_count):

        node_name = f"{site.lower()}{node_num+1}"
        node = slice.add_node(name=node_name, site=site, cores=8, ram=16, disk=100, image='docker_rocky_8')
        
        iface = node.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
        network = slice.get_network(name=f'{local_network_base_name}{site}')

        network.add_interface(iface)
        iface.set_mode('auto')
        node.add_route(subnet=IPv4Network('192.168.0.0/16'), next_hop=network.get_gateway())
        
        print(f"Adding control plane to node {node_name}")
        #add_control_plane(slice, node)
        node.add_fabnet()


        print(f"Getting  all_devs_template: {node_name}")

        all_devs_template = ''
        for iface in node.get_interfaces():
            all_devs_template +=  f" {{{{ interfaces['{iface.get_name()}'].dev }}}} "
        
        print(f"Adding post boot tasks: {node_name}")

        # Add post boot config    
        node.add_post_boot_upload_directory('node_tools','.')
        node.add_post_boot_execute('node_tools/enable_docker.sh {{ _self_.image }}')
        node.add_post_boot_upload_directory('docker_containers','.')
        node.add_post_boot_execute(f'cd docker_containers/fabric_multitool; docker compose up -d ')

#slice.save('frrouting.graphml')
print("submitting...")

slice.submit()

## Run the Experiment

We will just test `ping` RTT and look at `tracepath`. Your experiment should be more interesting!

Notice that if you run this quickly and repeatedly run this test against a specific target, you may see changes to the tracepath.  Initially the ping may even fail.  Why do you think this is happening?


In [ ]:
try:
    slice = fablib.get_slice(slice_name)

    
    source_node = slice.get_node(name=f'{sites[0].lower()}1')
    
    target_node = slice.get_node(name=f'{sites[1].lower()}1')
    target_ip=target_node.get_interface(network_name=f'net_local{sites[1]}').get_ip_addr()
    
    print(f"Testing target node: {target_node.get_name()}, target IP: {target_ip}")

    stdout, stderr = source_node.execute(f'ping -c 5 {target_ip}')

    stdout, stderr = source_node.execute(f'tracepath {target_ip}')
    
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
slice = fablib.get_slice(slice_name)

    
source_node = slice.get_node(name=f'{sites[0].lower()}1')

target_node = slice.get_node(name=f'{sites[1].lower()}1')
target_ip=target_node.get_interface(network_name=f'net_local{sites[1]}').get_ip_addr()

print(f"Testing target node: {target_node.get_name()}, target IP: {target_ip}")

stdout1, stderr1 = target_node.execute("docker run -d --rm "
                                "--network host "
                                "pruth/fabric-multitool-rockylinux8:latest "
                                "iperf3 -s -1"
                                , quiet=True, output_file=f"{target_node.get_name()}.log");

stdout2, stderr2 = source_node.execute("docker run --rm "
                                "--network host "
                                "pruth/fabric-multitool-rockylinux8:latest "
                                f"iperf3 -c {target_ip} -P 8 -t 60 -i 60 -O 10"
                                , quiet=False, output_file=f"{node.get_name()}.log");

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")